In [1]:
import pandas as pd
import requests
import csv
import re
import os

In [2]:
OMDB_API_KEY1 = 'http://www.omdbapi.com/?i=tt3896198&apikey=f4bb169d'
OMDB_API_KEY2 = 'http://www.omdbapi.com/?i=tt3896198&apikey=74a83470'
OMDB_API_KEY3 = 'http://www.omdbapi.com/?i=tt3896198&apikey=6775057e'
OMDB_API_KEY4 = 'http://www.omdbapi.com/?i=tt3896198&apikey=25e3d04'

In [3]:
OMDB_API_KEY1 = 'http://www.omdbapi.com/?i=tt3896198&apikey=a54cd785'
OMDB_API_KEY2 = 'http://www.omdbapi.com/?i=tt3896198&apikey=b4d8016'
OMDB_API_KEY3 = 'http://www.omdbapi.com/?i=tt3896198&apikey=babec89f'
OMDB_API_KEY4 = 'http://www.omdbapi.com/?i=tt3896198&apikey=ca17d525'

In [ ]:
API_KEYS = [OMDB_API_KEY1, OMDB_API_KEY2, OMDB_API_KEY3, OMDB_API_KEY4]
REQUESTS_PER_KEY = 1000
TMDB_API_URL = "http://www.omdbapi.com/"

# Load movies CSV
movies = pd.read_csv("movies.csv")
output_file = "enriched_movies.csv"
start_index = 0

# If the output file already exists, resume from where it left off
if os.path.exists(output_file):
    enriched_data = pd.read_csv(output_file)
    enriched_ids = set(enriched_data["movieId"].tolist())
    start_index = len(enriched_ids)  # Resume from the last saved movie
else:
    enriched_ids = set()  # Initialize as empty

# Function to fetch movie data using OMDb API
def fetch_movie_data(movie_id, title, year, api_key):
    params = {
        "apikey": api_key,
        "t": title,
        "y": year
    }
    try:
        request_url = api_key + "&t=" + title.replace(" ", "+") + "&y=" + str(year)
        response = requests.get(request_url)
        response.raise_for_status()
        data = response.json()
        
        if data.get("Response") == "True":
            plot = data.get("Plot", "No plot available")
            cast = data.get("Actors", "Cast information unavailable")
            runtime = data.get("Runtime", "Runtime information unavailable")
            director = data.get("Director", "Director information unavailable")
            awards = data.get("Awards", "Awards information unavailable")
            ratings = data.get("Ratings", "Ratings information unavailable")
            metascore = data.get("Metascore", "Metascore information unavailable")
            imdbRating = data.get("imdbRating", "imdbRating information unavailable")
            imdbVotes = data.get("imdbVotes", "imdbVotes information unavailable")
            boxOffice = data.get("BoxOffice", "BoxOffice information unavailable")
            reviews = ["Review data unavailable"] * 10  # Placeholder reviews
            return {
                "movieId": movie_id,
                "title": title,
                "year": year,
                "plot": plot,
                "director": director,
                "cast": cast,
                "awards": awards,
                "runtime": runtime,
                "ratings": ratings,
                "metascore": metascore,
                "imdbRating": imdbRating,
                "imdbVotes": imdbVotes,
                "boxOffice": boxOffice,
                **{f"review_{i+1}": reviews[i] for i in range(10)}
            }
        else:
            return None  # Handle missing data gracefully
    except requests.RequestException as e:
        print(f"Error fetching data for movie ID {movie_id}: {e}")
        return None

# Writing to CSV file function
def write_to_csv(data):
    fieldnames = ["movieId", "title", "year", "plot", "director", "cast", "awards", "runtime", "ratings", "metascore", "imdbRating", "imdbVotes", "boxOffice"] + [f"review_{i+1}" for i in range(10)]
    # Append if the file already exists, otherwise create and write headers
    mode = 'a' if os.path.exists(output_file) else 'w'
    with open(output_file, mode, newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if mode == 'w':
            writer.writeheader()  # Write header only once
        writer.writerows(data)

# Main script
daily_limit = REQUESTS_PER_KEY * len(API_KEYS)  # 4000 requests per day
api_key_index = 0  # Start with the first API key
collected_data = []
request_count = 0

for index, row in movies.iterrows():
    if index < start_index or row['movieId'] in enriched_ids:
        continue  # Skip already processed movies

    # Extract title and year
    title, year = row['title'], None
    if '(' in title and ')' in title:
        title, year = title.rsplit('(', 1)
        year = year[:-1]  # Remove closing parenthesis
    
    # Attempt fetching movie data
    data = fetch_movie_data(row['movieId'], title.strip(), year, API_KEYS[api_key_index])
    
    # Retry logic if data fetch fails
    retries = 3
    while data is None and retries > 0:
        data = fetch_movie_data(row['movieId'], title.strip(), year, API_KEYS[4 - retries])
        retries -= 1
    
    if data:
        collected_data.append(data)
        enriched_ids.add(row['movieId'])
        request_count += 1

    # Periodically write to CSV to save progress
    if len(collected_data) >= 100:
        write_to_csv(collected_data)
        collected_data = []  # Clear buffer after saving

# Final save for remaining data
if collected_data:
    write_to_csv(collected_data)

print("Data fetching completed. Saved to enriched_movies.csv.")

Error fetching data for movie ID 6237: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=babec89f&t=Glenn+Miller+Story,+The&y=1953
Error fetching data for movie ID 6427: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=ca17d525&t=Railway+Children,+The&y=1970
Error fetching data for movie ID 6609: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=babec89f&t=Gospel+According+to+St.+Matthew,+The+(Vangelo+secondo+Matteo,+Il)&y=1964
Error fetching data for movie ID 6783: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=b4d8016&t=Rules+of+the+Game,+The+(La+r%C3%A8gle+du+jeu)&y=1939
Error fetching data for movie ID 6918: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt3896198&apikey=a54cd785&t=Unvanquished,+The+(Aparajito)&y=1957
Error fetching data for movie ID 6971: 401 Client Error: Unauthorized for url: http://www.omdbapi.com/?i=tt389619

In [2]:
data = pd.read_csv("enriched_movies.csv")

In [11]:
data.columns

Index(['movieId', 'title', 'year', 'plot', 'director', 'cast', 'awards',
       'runtime', 'ratings', 'metascore', 'imdbRating', 'imdbVotes',
       'boxOffice', 'review_1', 'review_2', 'review_3', 'review_4', 'review_5',
       'review_6', 'review_7', 'review_8', 'review_9', 'review_10'],
      dtype='object')